# Proyecto

Uso de transformers para hacer traducción de texto


In [ ]:
# Run if necesary
# !pip install transformers datasets evaluate
# !pip install ipywidgets
# !pip install transformers datasets evaluate sacrebleu

In [3]:
from huggingface_hub import notebook_login

# notebook_login()

In [5]:
from datasets import load_dataset

books = load_dataset('opus_books', 'en-fr')

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
# Split the datset into a train and test set with the train test split method
# from sklearn.model_selection import train_test_split

books = books['train'].train_test_split(test_size=0.2)

In [8]:
# Then take a look at an example:
books['train'][1]

{'id': '78929',
 'translation': {'en': 'After these conventional phrases, which the impatient curiosity of M. Valenod found a way of cutting short, he learned the most mortifying things in the world for his own self-esteem.',
  'fr': 'Après ces phrases d’usage, que l’impatiente curiosité de M. Valenod trouva l’art d’abréger, il apprit les choses les plus mortifiantes pour son amour-propre.'}}

the next step is to load a T5 tokenizer to process the english-franch language pairs

In [ ]:
from transformers import AutoTokenizer

checkpoint = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

d:\Users\rod_e\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rod_e\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


The preprocessing function you want to create needs to:

1. Prefix the input with a prompt so T5 knows this is a translation task. Some models capable of multiple NLP tasks require prompting for specific tasks.
2. Tokenize the input (English) and target (French) separately because you can’t tokenize French text with a tokenizer pretrained on an English vocabulary.
3. Truncate sequences to be no longer than the maximum length set by the max_length parameter.

In [10]:
source_lang = 'en'
target_lang = 'fr'
prefix = 'translate English to French: '

In [11]:
def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples['translation']]
    targets = [example[target_lang] for example in examples['translation']]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

To apply the preprocessing function over the entire dataset, use 🤗 Datasets map method. You can speed up the map function by setting batched=True to process multiple elements of the dataset at once:

In [12]:
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/101668 [00:00<?, ? examples/s]

Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

In [13]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

# Evaluate
Including a metric during training is often helpful for evaluating your model's performance. You can qquickly load a evaluation methon with the Evaluate library. For this task, load the **sacreBLEU** metric (see the Evaluate quick tour to learn more about how to load and compute a metric)

In [14]:
import evaluate
metric = evaluate.load('sacrebleu')

Then create a function that passes your predictions and labels to compute to calculate the SacreBLEU score:

In [15]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip() for label in labels]]
    
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    labels = np.where(labels != 100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {'bleu' : result['score']}
    
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    
    result['gen_len'] = np.mean(prediction_lens)
    result = {'k' : round(v,4) for k, v in result.items()}
    return result

Your compute_metrics function is ready to go now, and you’ll return to it when you setup your training.

You're ready to start trainig your model now! Load T5 with automodelforseq2seqlm


In [19]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

ImportError: 
AutoModelForSeq2SeqLM requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.
